In [79]:
########################################################################抓SQL資料
#################連結SQL
import pandas as pd 
import pyodbc
#import eikon as ek  # the Eikon Python wrapper package
import numpy as np  # NumPy
#import cufflinks as cf  # Cufflinks
import configparser as cp
#API 存成 pd.dataframe
import math
from datetime import datetime
import matplotlib.pyplot as plt

server = 'W82700152\sqlexpress' 
database = 'Fund' 
username = 'CTBC_Research' 
password = 'Research!!' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
query = "SELECT [Date],[Ticker],[Value],[flag],[flag_tier2] FROM [Fund].[dbo].[Warning_D_2]"       
dic_df= pd.read_sql(query, cnxn)
dic_df
#存檔
writer=pd.ExcelWriter('C:/Users/Z00045186/Desktop/國家_economic model/flag.xlsx')
dic_df.to_excel(writer,'工作表1')
writer.save()

In [136]:
'''
##所有基本function_old

########################################################################拿取price資料
def get_price(subject='美國'):
    '''取得price data'''
    import pandas as pd
    import numpy as np
    import tqdm
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.style.use("ggplot")
    
    #參數設定
    #observe_period = 60
    #predict_period = 60
    #predict_cycle=1
    
    
    df=pd.read_excel('ETF_price.xlsx', sheet_name ='price',encoding='big5')
    df['Date']=pd.to_datetime(df['Date'])
    df=df.set_index(keys=['Date'])
    df.sort_values("Date",inplace=True)    
    #轉成set
    column_name=set(df.columns)  
    #保留資料長度>2400的
    #for name in column_name:
    #    if df[name].count()<2400:
    #        df=df.drop(columns=[name])
    
     
    #製作(國家~~ISIN) dictionary
    country_dic={ '中國A':'SHCOMP Index',
                  '東協':'MXSO Index',        
                  '美國':'SPX Index',
                  '泛太平洋':'MXAP Index',
                  '拉丁美洲':'MXLA Index',
                  '新興亞洲':'MXMS Index',
                  '新興歐洲':'MXMU Index',
                  '歐洲':'SX5P Index',
                 }                

              
    df_price=df[country_dic[subject]].dropna()
   
    #補上 predict_period 個假資料 (for 預測
    #dti=pd.date_range(price.index[-1]+pd.Timedelta(days=1),freq='D',periods=predict_period)
    #temp=pd.Series(index=dti)
    #price=pd.concat([price,temp])
    
   
    indexes = df_price.index    
    #Split traning and testing data
    #import datetime
    #price_train = price[indexes < datetime.datetime(2012,1,1)]
    #price_test = price[indexes >= datetime.datetime(2012,1,1)]    
    
    
    return (df_price,indexes)

########################################################################拿取features資料
def get_features():
    '''取得所有flag'''
    import pandas as pd
    import numpy as np
    import tqdm
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.style.use("ggplot")
    
    df=pd.read_excel('flag.xlsx', sheet_name ='工作表1',encoding='big5')
    df['Date']=pd.to_datetime(df['Date'])
    df=df.set_index(keys=['Date'])
    df.sort_values("Date",inplace=True)    

    facet=list(df['flag'].unique())   #5大構面
    
    return (df,facet)


########################################################################製作features & label 表
def create_features(df_price,df_features,shift):
    '''製作所有features'''
    import talib
    import numpy as np
    import pandas as pd

    ticker_list=list(df_features['Ticker'].unique())

    #做出dataframe
    dataset = pd.DataFrame({
    'return': df_price.shift(-40)/df_price > 1,   #N天後的報酬率, 是否>1  --->label
    })
    #逐一將feature插入dataframe
    for ticker in ticker_list:
        feature=df_features[df_features['Ticker']==ticker]['Value']
        dataset[ticker]=feature

        

    
    column_name=set(dataset.columns)  #轉成set
    column_name.remove('return')
    #30個月(max-min)化  
    #for variable in column_name: 
    #    dataset[variable]=(dataset[variable]-dataset[variable].rolling(625).min())/(dataset[variable].rolling(625).max()-dataset[variable].rolling(625).min()
    
    #diff化  
    #for variable in column_name: 
    #    dataset[variable]=(dataset[variable]-dataset[variable].shift(20))+(dataset[variable].shift(20)-dataset[variable].shift(40))+(dataset[variable].shift(40)-dataset[variable].shift(60))
    
    #取平均化  
    #for variable in column_name: 
    #    dataset[variable]=dataset[variable].rolling(20).mean()   
    
    
    #z-score化  
    for variable in column_name: 
        dataset[variable]=(dataset[variable]-dataset[variable].rolling(40).mean())/dataset[variable].rolling(40).std()
    
    #diff化  
    #for variable in column_name: 
    #    dataset[variable]=(dataset[variable]-dataset[variable].shift(20))/dataset[variable].rolling(20).std()   


    #計算技術指標
    sma = talib.SMA(df_price, timeperiod=6)
    wma = talib.WMA(df_price, timeperiod=6)
    mom = talib.MOM(df_price, timeperiod=6)
    k, d = talib.STOCH  (df_price, df_price, df_price, fastk_period=5, slowk_period=1, slowd_period=1)
    k2, d2 = talib.STOCH(df_price, df_price, df_price, fastk_period=10, slowk_period=10, slowd_period=1)
    k3, d3 = talib.STOCH(df_price, df_price, df_price, fastk_period=20, slowk_period=20, slowd_period=1)
    k4, d4 = talib.STOCH(df_price, df_price, df_price, fastk_period=40, slowk_period=40, slowd_period=1)
    k5, d5 = talib.STOCH(df_price, df_price, df_price, fastk_period=60, slowk_period=60, slowd_period=1)
    k6, d6 = talib.STOCH(df_price, df_price, df_price, fastk_period=120, slowk_period=120, slowd_period=1)
    k7, d7 = talib.STOCH(df_price, df_price, df_price, fastk_period=240, slowk_period=120, slowd_period=1)    
    rsi = talib.RSI (df_price, timeperiod=5)
    rsi2 = talib.RSI(df_price, timeperiod=10)
    rsi3 = talib.RSI(df_price, timeperiod=20)
    rsi4 = talib.RSI(df_price, timeperiod=40)
    rsi5 = talib.RSI(df_price, timeperiod=60)
    rsi6 = talib.RSI(df_price, timeperiod=120)
    rsi7 = talib.RSI(df_price, timeperiod=240)    
    macd1, macd2, macd3 = talib.MACD(df_price, fastperiod=6, slowperiod=3, signalperiod=3)
    willr = talib.WILLR(df_price, df_price, df_price, timeperiod=6)
    cci = talib.CCI(df_price, df_price, df_price, timeperiod=6)
    
    #技術指標
    technical = pd.DataFrame({
    'RSIb': rsi ,
    'RSIb2': rsi2 ,
    'RSIb3': rsi3 ,
    'RSIb4': rsi4 ,
    'RSIb5': rsi5 ,
    'RSIb6': rsi6 ,
    'RSIb7': rsi7 ,    
    'MOMb': mom - 0,
    'KDb': k - d,
    'KDb2': k2 - d2,
    'KDb3': k3 - d3,
    'KDb4': k4 - d4,
    'KDb5': k5 - d5,
    'KDb6': k6 - d6,
    'KDb7': k7 - d7,
    #
    'a5':   (df_price.rolling(5).mean()   / df_price),
    'a10':  (df_price.rolling(10).mean()  / df_price),
    'a20':  (df_price.rolling(20).mean()  / df_price),
    'a40':  (df_price.rolling(40).mean()  / df_price),
    'a80':  (df_price.rolling(80).mean()  / df_price),
    'a160': (df_price.rolling(160).mean() / df_price),
    'a320': (df_price.rolling(320).mean() / df_price),
#
    'b1': df_price/df_price.shift(5),
    'b2': df_price/df_price.shift(10),
    'b3': df_price/df_price.shift(20),
    'b4': df_price/df_price.shift(40),
    'b5': df_price/df_price.shift(60),
    'b6': df_price/df_price.shift(120),
    'b7': df_price/df_price.shift(240),
    'LINEARREG_SLOPE0': talib.LINEARREG_SLOPE(df_price, 5),
    'LINEARREG_SLOPE1': talib.LINEARREG_SLOPE(df_price, 10),

    'ADXR0': talib.ADXR(df_price, df_price, df_price, 5),
    'ADXR1': talib.ADXR(df_price, df_price, df_price, 10),
    'ADXR2': talib.ADXR(df_price, df_price, df_price, 20),
    'ADXR3': talib.ADXR(df_price, df_price, df_price, 40),
    'ADXR4': talib.ADXR(df_price, df_price, df_price, 60),
    'ADXR5': talib.ADXR(df_price, df_price, df_price, 120),    

    })
    
    dataset_new = dataset.join(technical)
    #dataset_new = dataset
    
    feature_names = list(dataset_new.columns[:-1])
    feature_names.remove('return')              #移除return, 否則train過程偷看答案
    
    return (dataset_new,feature_names)

In [303]:
'''
##所有基本function_純粹用Oracle (回測效果太差)

########################################################################拿取price資料
def get_price(subject='美國'):
    '''取得price data'''
    import pandas as pd
    import numpy as np
    import tqdm
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.style.use("ggplot")
    
    df=pd.read_excel('df_features.xlsx', sheet_name ='工作表1',encoding='big5')
    df['Date']=pd.to_datetime(df['Date'])
    df=df.set_index(keys=['Date'])
    df.sort_values("Date",inplace=True)  
    #參數設定
    #observe_period = 60
    #predict_period = 60
    #predict_cycle=1
    

    #轉成set
    #column_name=set(df.columns)  
    #保留資料長度>2400的
    #for name in column_name:
    #    if df[name].count()<2400:
    #        df=df.drop(columns=[name])
    
     
    #製作(國家~~ISIN) dictionary
    country_dic={ '中國A':'SHCOMP Index',
                  '東協':'MXSO Index',        
                  '美國':'SPX Index',
                  '泛太平洋':'MXAP Index',
                  '拉丁美洲':'MXLA Index',
                  '新興亞洲':'MXMS Index',
                  '新興歐洲':'MXMU Index',
                  '歐洲':'SX5P Index',
                 }                

              
    df_price=df[country_dic[subject]].dropna()
   
    #補上 predict_period 個假資料 (for 預測
    #dti=pd.date_range(price.index[-1]+pd.Timedelta(days=1),freq='D',periods=predict_period)
    #temp=pd.Series(index=dti)
    #price=pd.concat([price,temp])
    
   
    indexes = df_price.index    
   
    return (df_price,indexes)

########################################################################讀取features資料
def get_features():
    '''取得df_features'''
    import pandas as pd
    import numpy as np
    import tqdm
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.style.use("ggplot")
    
    df_features=pd.read_excel('df_features.xlsx', sheet_name ='工作表1',encoding='big5')
    df_features['Date']=pd.to_datetime(df_features['Date'])
    df_features=df_features.set_index(keys=['Date'])
    df_features.sort_values("Date",inplace=True)    
    
    list_features=list(df_features.columns.unique())   #
    
    return (df_features,list_features)
    


########################################################################製作features & label 表
def create_features(df_price,df_features,country,shift):
    '''製作所有features'''
    import talib
    import numpy as np
    import pandas as pd
    country_dic={ '中國A':'SHCOMP Index',
                  '東協':'MXSO Index',        
                  '美國':'SPX Index',
                  '泛太平洋':'MXAP Index',
                  '拉丁美洲':'MXLA Index',
                  '新興亞洲':'MXMS Index',
                  '新興歐洲':'MXMU Index',
                  '歐洲':'SX5P Index',
                 }                


    #做出dataframe
    #df_features['return']=df_price.shift(-40)/df_price > 1   #N天後的報酬率, 是否>1  --->label
    dataset=df_features
    dataset['return']=df_price.shift(-40)>df_price   #N天後的報酬率, 是否>1  --->label    
    
        

    
    column_name=set(dataset.columns)  #轉成set
    column_name.remove('return')
    #30個月(max-min)化  
    #for variable in column_name: 
    #    dataset[variable]=(dataset[variable]-dataset[variable].rolling(625).min())/(dataset[variable].rolling(625).max()-dataset[variable].rolling(625).min()
    
    #diff化  
    #for variable in column_name: 
    #    dataset[variable]=(dataset[variable]-dataset[variable].shift(20))+(dataset[variable].shift(20)-dataset[variable].shift(40))+(dataset[variable].shift(40)-dataset[variable].shift(60))
    
    #取平均化  
    #for variable in column_name: 
    #    dataset[variable]=dataset[variable].rolling(20).mean()   
    
    
    #z-score化  
    for variable in column_name: 
        dataset[variable]=(dataset[variable]-dataset[variable].rolling(40).mean())/dataset[variable].rolling(40).std()
    
    #diff化  
    #for variable in column_name: 
    #    dataset[variable]=(dataset[variable]-dataset[variable].shift(20))/dataset[variable].rolling(20).std()   


    #計算技術指標
    sma = talib.SMA(df_price, timeperiod=6)
    wma = talib.WMA(df_price, timeperiod=6)
    mom = talib.MOM(df_price, timeperiod=6)
    k, d = talib.STOCH  (df_price, df_price, df_price, fastk_period=5, slowk_period=1, slowd_period=1)
    k2, d2 = talib.STOCH(df_price, df_price, df_price, fastk_period=10, slowk_period=10, slowd_period=1)
    k3, d3 = talib.STOCH(df_price, df_price, df_price, fastk_period=20, slowk_period=20, slowd_period=1)
    k4, d4 = talib.STOCH(df_price, df_price, df_price, fastk_period=40, slowk_period=40, slowd_period=1)
    k5, d5 = talib.STOCH(df_price, df_price, df_price, fastk_period=60, slowk_period=60, slowd_period=1)
    k6, d6 = talib.STOCH(df_price, df_price, df_price, fastk_period=120, slowk_period=120, slowd_period=1)
    k7, d7 = talib.STOCH(df_price, df_price, df_price, fastk_period=240, slowk_period=120, slowd_period=1)    
    rsi = talib.RSI (df_price, timeperiod=5)
    rsi2 = talib.RSI(df_price, timeperiod=10)
    rsi3 = talib.RSI(df_price, timeperiod=20)
    rsi4 = talib.RSI(df_price, timeperiod=40)
    rsi5 = talib.RSI(df_price, timeperiod=60)
    rsi6 = talib.RSI(df_price, timeperiod=120)
    rsi7 = talib.RSI(df_price, timeperiod=240)    
    macd1, macd2, macd3 = talib.MACD(df_price, fastperiod=6, slowperiod=3, signalperiod=3)
    willr = talib.WILLR(df_price, df_price, df_price, timeperiod=6)
    cci = talib.CCI(df_price, df_price, df_price, timeperiod=6)
    
    #技術指標
    technical = pd.DataFrame({
    'RSIb': rsi ,
    'RSIb2': rsi2 ,
    'RSIb3': rsi3 ,
    'RSIb4': rsi4 ,
    'RSIb5': rsi5 ,
    'RSIb6': rsi6 ,
    'RSIb7': rsi7 ,    
    'MOMb': mom - 0,
    'KDb': k - d,
    'KDb2': k2 - d2,
    'KDb3': k3 - d3,
    'KDb4': k4 - d4,
    'KDb5': k5 - d5,
    'KDb6': k6 - d6,
    'KDb7': k7 - d7,
    #
    'a5':   (df_price.rolling(5).mean()   / df_price),
    'a10':  (df_price.rolling(10).mean()  / df_price),
    'a20':  (df_price.rolling(20).mean()  / df_price),
    'a40':  (df_price.rolling(40).mean()  / df_price),
    'a80':  (df_price.rolling(80).mean()  / df_price),
    'a160': (df_price.rolling(160).mean() / df_price),
    #'a320': (df_price.rolling(320).mean() / df_price),
#
    'b1': df_price/df_price.shift(5),
    'b2': df_price/df_price.shift(10),
    'b3': df_price/df_price.shift(20),
    'b4': df_price/df_price.shift(40),
    'b5': df_price/df_price.shift(60),
    'b6': df_price/df_price.shift(120),
    #'b7': df_price/df_price.shift(240),
    'LINEARREG_SLOPE0': talib.LINEARREG_SLOPE(df_price, 5),
    'LINEARREG_SLOPE1': talib.LINEARREG_SLOPE(df_price, 10),

    'ADXR0': talib.ADXR(df_price, df_price, df_price, 5),
    'ADXR1': talib.ADXR(df_price, df_price, df_price, 10),
    'ADXR2': talib.ADXR(df_price, df_price, df_price, 20),
    'ADXR3': talib.ADXR(df_price, df_price, df_price, 40),
    'ADXR4': talib.ADXR(df_price, df_price, df_price, 60),
    'ADXR5': talib.ADXR(df_price, df_price, df_price, 120),    

    })
    
    dataset_new = dataset.join(technical)
    #dataset_new = dataset
    
    feature_names = list(dataset_new.columns[:-1])
    feature_names.remove('return')              #移除return, 否則train過程偷看答案
    
    return (dataset_new,feature_names)

In [73]:

##所有基本function_(SQL+Oracle)

########################################################################拿取price資料
def get_price(subject='美國'):
    '''取得price data'''
    import pandas as pd
    import numpy as np
    import tqdm
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.style.use("ggplot")
    
    #參數設定
    #observe_period = 60
    #predict_period = 60
    #predict_cycle=1
    
    
    df=pd.read_excel('ETF_price.xlsx', sheet_name ='price',encoding='big5')
    df['Date']=pd.to_datetime(df['Date'])
    df=df.set_index(keys=['Date'])
    df.sort_values("Date",inplace=True)    
    #轉成set
    column_name=set(df.columns)  
    #保留資料長度>2400的
    #for name in column_name:
    #    if df[name].count()<2400:
    #        df=df.drop(columns=[name])
    
     
    #製作(國家~~ISIN) dictionary
    country_dic={ '中國A':'SHCOMP Index',
                  '東協':'MXSO Index',        
                  '美國':'SPX Index',
                  '泛太平洋':'MXAP Index',
                  '拉丁美洲':'MXLA Index',
                  '新興亞洲':'MXMS Index',
                  '新興歐洲':'MXMU Index',
                  '歐洲':'SX5P Index',
                 }                

              
    df_price=df[country_dic[subject]].dropna()
   
    #補上 predict_period 個假資料 (for 預測
    #dti=pd.date_range(price.index[-1]+pd.Timedelta(days=1),freq='D',periods=predict_period)
    #temp=pd.Series(index=dti)
    #price=pd.concat([price,temp])
    
   
    indexes = df_price.index    
    #Split traning and testing data
    #import datetime
    #price_train = price[indexes < datetime.datetime(2012,1,1)]
    #price_test = price[indexes >= datetime.datetime(2012,1,1)]    
    
    
    return (df_price,indexes)

########################################################################拿取features資料_flag
def get_features_flag():
    '''取得所有flag'''
    import pandas as pd
    import numpy as np
    import tqdm
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.style.use("ggplot")
    
    df_features_SQL=pd.read_excel('flag.xlsx', sheet_name ='工作表1',encoding='big5')
    df_features_SQL['Date']=pd.to_datetime(df_features_SQL['Date'])
    df_features_SQL=df_features_SQL.set_index(keys=['Date'])
    df_features_SQL.sort_values("Date",inplace=True)    

    facet=list(df_features_SQL['flag'].unique())   #5大構面
    
    
    return df_features_SQL

########################################################################拿取features資料_oracle
def get_features_oracle():
    '''取得所有flag'''
    import pandas as pd
    import numpy as np
    import tqdm
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.style.use("ggplot")
    
    df_features_oracle=pd.read_excel('oracle.xlsx', sheet_name ='工作表1',encoding='big5')
    df_features_oracle['Date']=pd.to_datetime(df_features_oracle['Date'])
    df_features_oracle=df_features_oracle.set_index(keys=['Date'])
    df_features_oracle.sort_values("Date",inplace=True)    

    facet=list(df_features_oracle.columns.unique())   #

    
    return df_features_oracle

########################################################################合併原始features檔案
'''
def get_features_all(features_SQL,features_oracle):
    
    import pandas as pd
    import numpy as np
    import tqdm
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.style.use("ggplot")
    
    df_features=features_SQL.append(features_oracle)


    features_name=list(df.columns.unique())   #
    
    return (df_features,features_name)
'''    
########################################################################製作features & label 表
def create_features(df_price,df_features_SQL,shift):
    '''製作所有features'''
    import talib
    import numpy as np
    import pandas as pd

    ticker_list=list(df_features_SQL['Ticker'].unique())

    #做出dataframe
    dataset = pd.DataFrame({
    'return': df_price.shift(-40)/df_price > 1,   #N天後的報酬率, 是否>1  --->label
    })
    #逐一將feature插入dataframe
    for ticker in ticker_list:
        feature=df_features_SQL[df_features_SQL['Ticker']==ticker]['Value']
        dataset[ticker]=feature

    #結合oracle抓的features    
    #dataset=dataset.append(df_features_oracle)    

    
    column_name=set(dataset.columns)  #轉成set
    column_name.remove('return')
    #30個月(max-min)化  
    #for variable in column_name: 
    #    dataset[variable]=(dataset[variable]-dataset[variable].rolling(625).min())/(dataset[variable].rolling(625).max()-dataset[variable].rolling(625).min()
    
    #diff化  
    #for variable in column_name: 
    #    dataset[variable]=(dataset[variable]-dataset[variable].shift(20))+(dataset[variable].shift(20)-dataset[variable].shift(40))+(dataset[variable].shift(40)-dataset[variable].shift(60))
    
    #取平均化  
    #for variable in column_name: 
    #    dataset[variable]=dataset[variable].rolling(20).mean()   
    
    
    #z-score化  
    for variable in column_name: 
        dataset[variable]=(dataset[variable]-dataset[variable].rolling(40).mean())/dataset[variable].rolling(40).std()
    
    #diff化  
    #for variable in column_name: 
    #    dataset[variable]=(dataset[variable]-dataset[variable].shift(20))/dataset[variable].rolling(20).std()   


    #計算技術指標
    sma = talib.SMA(df_price, timeperiod=6)
    wma = talib.WMA(df_price, timeperiod=6)
    mom = talib.MOM(df_price, timeperiod=6)
    k, d = talib.STOCH  (df_price, df_price, df_price, fastk_period=5, slowk_period=1, slowd_period=1)
    k2, d2 = talib.STOCH(df_price, df_price, df_price, fastk_period=10, slowk_period=10, slowd_period=1)
    k3, d3 = talib.STOCH(df_price, df_price, df_price, fastk_period=20, slowk_period=20, slowd_period=1)
    k4, d4 = talib.STOCH(df_price, df_price, df_price, fastk_period=40, slowk_period=40, slowd_period=1)
    k5, d5 = talib.STOCH(df_price, df_price, df_price, fastk_period=60, slowk_period=60, slowd_period=1)
    k6, d6 = talib.STOCH(df_price, df_price, df_price, fastk_period=120, slowk_period=120, slowd_period=1)
    k7, d7 = talib.STOCH(df_price, df_price, df_price, fastk_period=240, slowk_period=120, slowd_period=1) 
    rsi = talib.RSI (df_price, timeperiod=5)
    rsi2 = talib.RSI(df_price, timeperiod=10)
    rsi3 = talib.RSI(df_price, timeperiod=20)
    rsi4 = talib.RSI(df_price, timeperiod=40)
    rsi5 = talib.RSI(df_price, timeperiod=60)
    rsi6 = talib.RSI(df_price, timeperiod=120)
    rsi7 = talib.RSI(df_price, timeperiod=240)    
    macd1, macd2, macd3 = talib.MACD(df_price, fastperiod=6, slowperiod=3, signalperiod=3)
    willr = talib.WILLR(df_price, df_price, df_price, timeperiod=6)
    cci = talib.CCI(df_price, df_price, df_price, timeperiod=6)
    
    #技術指標
    technical = pd.DataFrame({
    'RSIb': rsi ,
    'RSIb2': rsi2 ,
    'RSIb3': rsi3 ,
    'RSIb4': rsi4 ,
    'RSIb5': rsi5 ,
    'RSIb6': rsi6 ,
    'RSIb7': rsi7 ,    
    'MOMb': mom - 0,
    'KDb': k - d,
    'KDb2': k2 - d2,
    'KDb3': k3 - d3,
    'KDb4': k4 - d4,
    'KDb5': k5 - d5,
    'KDb6': k6 - d6,
    'KDb7': k7 - d7,
    
    'a5':   (df_price.rolling(5).mean()   / df_price),
    'a10':  (df_price.rolling(10).mean()  / df_price),
    'a20':  (df_price.rolling(20).mean()  / df_price),
    'a40':  (df_price.rolling(40).mean()  / df_price),
    'a80':  (df_price.rolling(80).mean()  / df_price),
    'a160': (df_price.rolling(160).mean() / df_price),
    #'a320': (df_price.rolling(320).mean() / df_price),

    'b1': df_price/df_price.shift(5),
    'b2': df_price/df_price.shift(10),
    'b3': df_price/df_price.shift(20),
    'b4': df_price/df_price.shift(40),
    'b5': df_price/df_price.shift(60),
    'b6': df_price/df_price.shift(120),
    'b7': df_price/df_price.shift(240),
    'LINEARREG_SLOPE0': talib.LINEARREG_SLOPE(df_price, 5),
    'LINEARREG_SLOPE1': talib.LINEARREG_SLOPE(df_price, 10),

    'ADXR0': talib.ADXR(df_price, df_price, df_price, 5),
    'ADXR1': talib.ADXR(df_price, df_price, df_price, 10),
    'ADXR2': talib.ADXR(df_price, df_price, df_price, 20),
    'ADXR3': talib.ADXR(df_price, df_price, df_price, 40),
    'ADXR4': talib.ADXR(df_price, df_price, df_price, 60),
    'ADXR5': talib.ADXR(df_price, df_price, df_price, 120),    

    })
    
    dataset_new = dataset.join(technical)
    #dataset_new = dataset.dropna(how='any',axis=1) 
    
    feature_names = list(dataset_new.columns[:-1])
    feature_names.remove('return')              #移除return, 否則train過程偷看答案
    #dataset_new = dataset_new.dropna(how='any',axis=1) 
    
    return (dataset_new,feature_names)



In [ ]:
'''
##所有基本function_(SQL+Oracle)

########################################################################拿取price資料
def get_price(subject='美國'):
    '''取得price data'''
    import pandas as pd
    import numpy as np
    import tqdm
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.style.use("ggplot")
    
    #參數設定
    #observe_period = 60
    #predict_period = 60
    #predict_cycle=1
    
    
    df=pd.read_excel('ETF_price.xlsx', sheet_name ='price',encoding='big5')
    df['Date']=pd.to_datetime(df['Date'])
    df=df.set_index(keys=['Date'])
    df.sort_values("Date",inplace=True)    
    #轉成set
    column_name=set(df.columns)  
    #保留資料長度>2400的
    #for name in column_name:
    #    if df[name].count()<2400:
    #        df=df.drop(columns=[name])
    
     
    #製作(國家~~ISIN) dictionary
    country_dic={ '中國A':'SHCOMP Index',
                  '東協':'MXSO Index',        
                  '美國':'SPX Index',
                  '泛太平洋':'MXAP Index',
                  '拉丁美洲':'MXLA Index',
                  '新興亞洲':'MXMS Index',
                  '新興歐洲':'MXMU Index',
                  '歐洲':'SX5P Index',
                 }                

              
    df_price=df[country_dic[subject]].dropna()
   
    #補上 predict_period 個假資料 (for 預測
    #dti=pd.date_range(price.index[-1]+pd.Timedelta(days=1),freq='D',periods=predict_period)
    #temp=pd.Series(index=dti)
    #price=pd.concat([price,temp])
    
   
    indexes = df_price.index    
    #Split traning and testing data
    #import datetime
    #price_train = price[indexes < datetime.datetime(2012,1,1)]
    #price_test = price[indexes >= datetime.datetime(2012,1,1)]    
    
    
    return (df_price,indexes)

########################################################################拿取features資料_flag
def get_features_flag():
    '''取得所有flag'''
    import pandas as pd
    import numpy as np
    import tqdm
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.style.use("ggplot")
    
    df_features_SQL=pd.read_excel('flag.xlsx', sheet_name ='工作表1',encoding='big5')
    df_features_SQL['Date']=pd.to_datetime(df_features_SQL['Date'])
    df_features_SQL=df_features_SQL.set_index(keys=['Date'])
    df_features_SQL.sort_values("Date",inplace=True)    

    facet=list(df_features_SQL['flag'].unique())   #5大構面
    
    
    return df_features_SQL

########################################################################拿取features資料_oracle
def get_features_oracle():
    '''取得所有flag'''
    import pandas as pd
    import numpy as np
    import tqdm
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.style.use("ggplot")
    
    df_features_oracle=pd.read_excel('oracle.xlsx', sheet_name ='工作表1',encoding='big5')
    df_features_oracle['Date']=pd.to_datetime(df_features_oracle['Date'])
    df_features_oracle=df_features_oracle.set_index(keys=['Date'])
    df_features_oracle.sort_values("Date",inplace=True)    

    facet=list(df_features_oracle.columns.unique())   #

    
    return df_features_oracle

########################################################################製作features & label 表
#def create_features(df_price,df_features_SQL,shift):
#    '''製作所有features'''
#    import talib
#    import numpy as np
#    import pandas as pd
#
#    ticker_list=list(df_features_SQL['Ticker'].unique())


'''

In [69]:
'''#測試
country='中國A'
df_features_SQL=get_features_flag()
(df_price,indexes)=get_price(subject=country)

'''

In [ ]:
'''
import talib
import numpy as np
import pandas as pd

ticker_list=list(df_features_SQL['Ticker'].unique())

#做出dataframe
dataset = pd.DataFrame({
'return': df_price.shift(-40)/df_price > 1,   #N天後的報酬率, 是否>1  --->label
})
#逐一將feature插入dataframe
for ticker in ticker_list:
    feature=df_features_SQL[df_features_SQL['Ticker']==ticker]['Value']
    dataset[ticker]=feature
    
df_features_SQL
dataset
'''

In [20]:
'''
(df_price,indexes)=get_price(subject='美國')
df_features_SQL=get_features_flag()
df_features_oracle=get_features_oracle()
(dataset,feature_names)=create_features(df_price,df_features_SQL,df_features_oracle,shift=40)


In [ ]:
'''dataset_new.tail(7)

In [208]:
'''
########################################################################訓練 lightgbm  (分5個平行世界投票) (固定世界) (只回測一年) (rank) 最後簡報版

def multiple_choose_rank_model(period_rolling=3):
    '''分5個平行世界投票'''
    import pandas as pd
    import numpy as np
    import lightgbm as lgb
    import matplotlib.pyplot as plt
    import seaborn as sns
    %matplotlib inline
    
    country_dic={'中國A':'SHCOMP Index',
                 '東協':'MXSO Index',
                 '美國':'SPX Index',
                 '泛太平洋':'MXAP Index',
                 '拉丁美洲':'MXLA Index',
                 '新興亞洲':'MXMS Index',
                 '新興歐洲':'MXMU Index',
                 '歐洲':'SX5P Index',
                    }
    
    #創建空白dataframe
    df_outcome=pd.DataFrame(columns=[n for n in country_dic.keys()])
    df_proba=pd.DataFrame(columns=[n for n in country_dic.keys()])
    
    #參數:
    priority_number=50  #取前n個importance的feature
    predict_period=40  #預測天數  
    
    
    #train period (固定)    
    period=[x  for x in range(2007,2017,1)]
    save=[None]*20
    for x in range(0,len(period)-period_rolling,period_rolling):
        save[x]=[period[x],period[x+period_rolling-1],period[x+period_rolling-1]]
    save2 = list(filter(None, save)) 
    
    


    #model dict
    model_dic={}   
    for x in country_dic.keys():
        model_dic[x]=[None]*len(save2)          
        
    ###第1次train:建構平行世界,存取model
    for x in save2:
        date_startbacktest=str(x[0])
        date_endbacktest=str(x[1])
        date_test=str(x[2])
        print(date_startbacktest,date_endbacktest,date_test)
        train_start=date_startbacktest
        train_end=date_endbacktest
        test_start=date_test
        test_end=date_test
        
        
        (df_features,facet)=get_features()
        df_features=df_features['2006':'2020/12/23']
        df_features=df_features.dropna(how='any',axis=1) 
        for country in country_dic.keys():
            print('開始訓練'+country)
            df_features
            gbm = lgb.LGBMClassifier(n_estimators=1000, random_state=5, learning_rate=0.05)
            (df_price,indexes)=get_price(df_features=df_features,subject=country)
            (dataset,feature_names)=create_features(df_price=df_price,df_features=df_features,shift=predict_period)    

            
            dataset_train = dataset.loc[train_start:train_end]
            dataset_test = dataset.loc[test_start:test_end]
            
            train = dataset_train[feature_names], dataset_train['return']
            test = dataset_test[feature_names], dataset_test['return']
            
            gbm.fit(*train)
            model_dic[country][save2.index(x)]=gbm
            


            
######################################################
    #test period  
    period_test=pd.period_range('1/1/2016','12/23/2020',freq='M')
    #選中的model dict
    model_choose_dic={}   
    for x in country_dic.keys():
        model_choose_dic[x]=None
        

    
    print('     ')
    print('##########開始選model並test data:')
    ###開始樣本外choose  &  test
    count=0 
    for i in range(14,len(period_test),1):

        print("測試區間{0}  {1}".format(period_test[i].start_time,period_test[i].end_time))

        test_start=period_test[i].start_time
        test_end=period_test[i].end_time        
        (df_features,facet)=get_features()
        df_features=df_features['2006':'2020/12/23']
        df_features=df_features.dropna(how='any',axis=1) 
        

        for country in country_dic.keys():
            

            
            score_list=[-100]
            for model in model_dic[country]:
                gbm = model     #挑出其中一個model
                (df_price,indexes)=get_price(df_features=df_features,subject=country)
                (dataset,feature_names)=create_features(df_price=df_price,df_features=df_features,shift=predict_period)
 
                
                #過去1個月回測score
                test_start=period_test[i-3].start_time
                test_end=period_test[i-3].end_time
                
                dataset_test = dataset.loc[test_start:test_end]
                test = dataset_test[feature_names], dataset_test['return']                
                score1=gbm.score(*test)
                
                
                #過去3個月回測score
                test_start=period_test[i-5].start_time
                test_end=period_test[i-3].end_time
                
                dataset_test = dataset.loc[test_start:test_end]
                test = dataset_test[feature_names], dataset_test['return']                
                score2=gbm.score(*test)                
                
                
                #過去6個月回測score
                test_start=period_test[i-8].start_time
                test_end=period_test[i-3].end_time
                
                dataset_test = dataset.loc[test_start:test_end]
                test = dataset_test[feature_names], dataset_test['return']                
                score3=gbm.score(*test)                  
                
                
                #過去12個月回測score
                test_start=period_test[i-14].start_time
                test_end=period_test[i-3].end_time
                
                dataset_test = dataset.loc[test_start:test_end]
                test = dataset_test[feature_names], dataset_test['return']                
                score4=gbm.score(*test)                   
                
                
                
                score_sum=score2+score3+score4
                
            
                if score_sum>=max(score_list):
                    model_choose_dic[country]=gbm
     
                score_list.append(score_sum)
            
            
            #用選出的model預測t的data
            gbm = model_choose_dic[country]
            test_start=period_test[i].start_time
            test_end=period_test[i].end_time
  
                
            dataset_test = dataset.loc[test_start:test_end] 
            test = dataset_test[feature_names], dataset_test['return']
            print("{0}的score為 {1}".format(country,gbm.score(*test)))
            
            #存成dataframe~score
            df_outcome.loc[count,country]=round(gbm.score(*test),4)
            df_outcome.loc[count,'date']=test_start
            
            #存成dataframe~proba     
            df_proba.loc[count,country]=gbm.predict_proba(dataset_test[feature_names])[-1][1]
            df_proba.loc[count,'date']=test_start            
    
        count=count+1
    
    #存檔
    writer=pd.ExcelWriter('C:/Users/Z00045186/Desktop/國家_economic model/multiple_choose_rank_model(period_rolling=3).xlsx')
    df_outcome.to_excel(writer,'outcome')
    df_proba.to_excel(writer,'proba')
    writer.save()
    return df_outcome


In [ ]:
df_outcome = multiple_choose_rank_model(period_rolling=3)

In [ ]:
'''
########################################################################訓練 lightgbm  (分5個平行世界投票) (固定世界) (只回測一年) (rank) 

period_rolling=3
'''分5個平行世界投票'''
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
    
country_dic={'中國A':'SHCOMP Index',
                 '東協':'MXSO Index',
                 '美國':'SPX Index',
                 '泛太平洋':'MXAP Index',
                 '拉丁美洲':'MXLA Index',
                 '新興亞洲':'MXMS Index',
                 '新興歐洲':'MXMU Index',
                 '歐洲':'SX5P Index',
                    }
    
#創建空白dataframe
df_outcome=pd.DataFrame(columns=[n for n in country_dic.keys()])
df_proba=pd.DataFrame(columns=[n for n in country_dic.keys()])
    
#參數:
priority_number=50  #取前n個importance的feature
predict_period=40  #預測天數  
    
    
#train period (固定)    
period=[x  for x in range(2007,2017,1)]
save=[None]*20
for x in range(0,len(period)-period_rolling,period_rolling):
    save[x]=[period[x],period[x+period_rolling-1],period[x+period_rolling-1]]
save2 = list(filter(None, save)) 
    
    


#model dict
model_dic={}   
for x in country_dic.keys():
    model_dic[x]=[None]*len(save2)          
        
###第1次train:建構平行世界,存取model
for x in save2:
    date_startbacktest=str(x[0])
    date_endbacktest=str(x[1])
    date_test=str(x[2])
    print(date_startbacktest,date_endbacktest,date_test)
    train_start=date_startbacktest
    train_end=date_endbacktest
    test_start=date_test
    test_end=date_test
        
        
    (df_features,facet)=get_features()
    df_features=df_features['2006':'2020/9/30']
    #df_features=df_features.dropna(how='any',axis=1) 
    for country in country_dic.keys():
        print('開始訓練'+country)
        df_features
        gbm = lgb.LGBMClassifier(n_estimators=1000, random_state=5, learning_rate=0.05)
        (df_price,indexes)=get_price(subject=country)
        (dataset,feature_names)=create_features(df_price,df_features,country,shift=predict_period)    
        dataset=dataset['2006':'2020/9/30']
        #dataset=dataset.dropna(how='any',axis=1) 
        df_price=df_price['2006':'2020/9/30']
        #df_price=df_price.dropna(how='any') 
        
        dataset_train = dataset.loc[train_start:train_end]
        dataset_test = dataset.loc[test_start:test_end]
            
        train = dataset_train[feature_names], dataset_train['return'].astype('int') ###
        test = dataset_test[feature_names], dataset_test['return']
            
        gbm.fit(*train)
        model_dic[country][save2.index(x)]=gbm
            


            
######################################################
#test period  
period_test=pd.period_range('1/1/2016','9/30/2020',freq='M')
#選中的model dict
model_choose_dic={}   
for x in country_dic.keys():
    model_choose_dic[x]=None
        

    
print('     ')
print('##########開始選model並test data:')
###開始樣本外choose  &  test
count=0 
for i in range(14,len(period_test),1):

    print("測試區間{0}  {1}".format(period_test[i].start_time,period_test[i].end_time))

    test_start=period_test[i].start_time
    test_end=period_test[i].end_time        
    (df_features,facet)=get_features()
    df_features=df_features['2006':'2020/9/30']
    #df_features=df_features.dropna(how='any',axis=1) 
        

    for country in country_dic.keys():
            

            
        score_list=[-100]
        for model in model_dic[country]:
            gbm = model     #挑出其中一個model
            (df_price,indexes)=get_price(subject=country)
            (dataset,feature_names)=create_features(df_price,df_features,country,shift=predict_period)
            dataset=dataset['2006':'2020/9/30']
            #dataset=dataset.dropna(how='any',axis=1) 
            df_price=df_price['2006':'2020/9/30']
            #df_price=df_price.dropna(how='any') 
                
            #過去1個月回測score
            test_start=period_test[i-3].start_time
            test_end=period_test[i-3].end_time
                
            dataset_test = dataset.loc[test_start:test_end]
            test = dataset_test[feature_names], dataset_test['return']                
            score1=gbm.score(*test)
                
                
            #過去3個月回測score
            test_start=period_test[i-5].start_time
            test_end=period_test[i-3].end_time
                
            dataset_test = dataset.loc[test_start:test_end]
            test = dataset_test[feature_names], dataset_test['return']                
            score2=gbm.score(*test)                
                
                
            #過去6個月回測score
            test_start=period_test[i-8].start_time
            test_end=period_test[i-3].end_time
                
            dataset_test = dataset.loc[test_start:test_end]
            test = dataset_test[feature_names], dataset_test['return']                
            score3=gbm.score(*test)                  
                
                
            #過去12個月回測score
            test_start=period_test[i-14].start_time
            test_end=period_test[i-3].end_time
                
            dataset_test = dataset.loc[test_start:test_end]
            test = dataset_test[feature_names], dataset_test['return']                
            score4=gbm.score(*test)                   
                
                
                
            score_sum=score2+score3+score4
                
            
            if score_sum>=max(score_list):
                model_choose_dic[country]=gbm
     
            score_list.append(score_sum)
            
            
        #用選出的model預測t的data
        gbm = model_choose_dic[country]
        test_start=period_test[i].start_time
        test_end=period_test[i].end_time
  
                
        dataset_test = dataset.loc[test_start:test_end] 
        test = dataset_test[feature_names], dataset_test['return']
        print("{0}的score為 {1}".format(country,gbm.score(*test)))
            
        #存成dataframe~score
        df_outcome.loc[count,country]=round(gbm.score(*test),4)
        df_outcome.loc[count,'date']=test_start
            
        #存成dataframe~proba     
        df_proba.loc[count,country]=gbm.predict_proba(dataset_test[feature_names])[-1][1]
        df_proba.loc[count,'date']=test_start            
    
    count=count+1
    
#存檔
writer=pd.ExcelWriter('C:/Users/Z00045186/Desktop/國家_economic model/multiple_choose_rank_model(period_rolling=3).xlsx')
df_outcome.to_excel(writer,'outcome')
df_proba.to_excel(writer,'proba')
writer.save()



In [95]:
########################################################################訓練 lightgbm  (分5個平行世界投票) (固定世界) (只回測一年) (rank) 例行產出版

def multiple_choose_rank_model(period_rolling=3):
    '''分5個平行世界投票'''
    import pandas as pd
    import numpy as np
    import lightgbm as lgb
    import matplotlib.pyplot as plt
    import seaborn as sns
    %matplotlib inline
    
    country_dic={'中國A':'SHCOMP Index',
                 '東協':'MXSO Index',
                 '美國':'SPX Index',
                 '泛太平洋':'MXAP Index',
                 '拉丁美洲':'MXLA Index',
                 '新興亞洲':'MXMS Index',
                 '新興歐洲':'MXMU Index',
                 '歐洲':'SX5P Index',
                    }
    
    #創建空白dataframe
    df_outcome=pd.DataFrame(columns=[n for n in country_dic.keys()])
    df_proba=pd.DataFrame(columns=[n for n in country_dic.keys()])
    
    #參數:
    priority_number=50  #取前n個importance的feature
    predict_period=40  #預測天數  
    
    
    #train period (固定)    
    period=[x  for x in range(2007,2017,1)]
    save=[None]*20
    for x in range(0,len(period)-period_rolling,period_rolling):
        save[x]=[period[x],period[x+period_rolling-1],period[x+period_rolling-1]]
    save2 = list(filter(None, save)) 
    
    


    #model dict
    model_dic={}   
    for x in country_dic.keys():
        model_dic[x]=[None]*len(save2)          
        
    ###第1次train:建構平行世界,存取model
    for x in save2:
        date_startbacktest=str(x[0])
        date_endbacktest=str(x[1])
        date_test=str(x[2])
        print(date_startbacktest,date_endbacktest,date_test)
        train_start=date_startbacktest
        train_end=date_endbacktest
        test_start=date_test
        test_end=date_test
        
        
        
        df_features_SQL=get_features_flag()
        #df_features_SQL=df_features_SQL['2006/1/2':'2020/12/25']
        #df_features_SQL=df_features_SQL.dropna(how='any',axis=1) 

        for country in country_dic.keys():
            print('開始訓練'+country)
       
            gbm = lgb.LGBMClassifier(n_estimators=1000, random_state=5, learning_rate=0.05)
            (df_price,indexes)=get_price(subject=country)
            (dataset,feature_names)=create_features(df_price=df_price,df_features_SQL=df_features_SQL,shift=predict_period)    
                                 
            
            dataset_train = dataset.loc[train_start:train_end]
            dataset_test = dataset.loc[test_start:test_end]
            
            train = dataset_train[feature_names], dataset_train['return']
            test = dataset_test[feature_names], dataset_test['return']
            
            gbm.fit(*train)
            model_dic[country][save2.index(x)]=gbm
            


            
######################################################
    #test period  
    period_test=pd.period_range('1/1/2016','1/4/2021',freq='M')
    #選中的model dict
    model_choose_dic={}   
    for x in country_dic.keys():
        model_choose_dic[x]=None
        

    
    print('     ')
    print('##########開始選model並test data:')
    ###開始樣本外choose  &  test
    count=0 
    for i in range(14,len(period_test),1):

        print("測試區間{0}  {1}".format(period_test[i].start_time,period_test[i].end_time))

        test_start=period_test[i].start_time
        test_end=period_test[i].end_time        
        
        df_features_SQL=get_features_flag()
        #df_features_SQL=df_features_SQL['2006':'2020/12/25']
        #df_features_SQL=df_features_SQL.dropna(how='any',axis=1) 
      

        for country in country_dic.keys():
            

            
            score_list=[-100]
            for model in model_dic[country]:
                gbm = model     #挑出其中一個model
                (df_price,indexes)=get_price(subject=country)
                (dataset,feature_names)=create_features(df_price=df_price,df_features_SQL=df_features_SQL,shift=predict_period)    
                
                #過去1個月回測score
                test_start=period_test[i-3].start_time
                test_end=period_test[i-3].end_time
                
                dataset_test = dataset.loc[test_start:test_end]
                test = dataset_test[feature_names], dataset_test['return']                
                score1=gbm.score(*test)
                
                
                #過去3個月回測score
                test_start=period_test[i-5].start_time
                test_end=period_test[i-3].end_time
                
                dataset_test = dataset.loc[test_start:test_end]
                test = dataset_test[feature_names], dataset_test['return']                
                score2=gbm.score(*test)                
                
                
                #過去6個月回測score
                test_start=period_test[i-8].start_time
                test_end=period_test[i-3].end_time
                
                dataset_test = dataset.loc[test_start:test_end]
                test = dataset_test[feature_names], dataset_test['return']                
                score3=gbm.score(*test)                  
                
                
                #過去12個月回測score
                test_start=period_test[i-14].start_time
                test_end=period_test[i-3].end_time
                
                dataset_test = dataset.loc[test_start:test_end]
                test = dataset_test[feature_names], dataset_test['return']                
                score4=gbm.score(*test)                   
                
                
                
                score_sum=score2+score3+score4
                
            
                if score_sum>=max(score_list):
                    model_choose_dic[country]=gbm
     
                score_list.append(score_sum)
            
            
            #用選出的model預測t的data
            gbm = model_choose_dic[country]
            #gbm = model_dic[country][0]   #檢查測試用code
            test_start=period_test[i].start_time
            test_end=period_test[i].end_time
  
                
            dataset_test = dataset.loc[test_start:test_end] 
            test = dataset_test[feature_names], dataset_test['return']
            print("{0}的score為 {1}".format(country,gbm.score(*test)))
            
            #存成dataframe~score
            df_outcome.loc[count,country]=round(gbm.score(*test),4)
            df_outcome.loc[count,'date']=test_start
            
            #存成dataframe~proba     
            df_proba.loc[count,country]=gbm.predict_proba(dataset_test[feature_names])[-1][1]
            df_proba.loc[count,'date']=test_start            
    
        count=count+1
    
    #存檔
    writer=pd.ExcelWriter('C:/Users/Z00045186/Desktop/國家_economic model/multiple_choose_rank_model(period_rolling=3).xlsx')
    df_outcome.to_excel(writer,'outcome')
    df_proba.to_excel(writer,'proba')
    writer.save()
    return (df_outcome,dataset,model_dic)


In [96]:




(df_outcome,dataset,model_dic) = multiple_choose_rank_model(period_rolling=3)




2007 2009 2009
開始訓練中國A
開始訓練東協
開始訓練美國
開始訓練泛太平洋
開始訓練拉丁美洲
開始訓練新興亞洲
開始訓練新興歐洲
開始訓練歐洲
2010 2012 2012
開始訓練中國A
開始訓練東協
開始訓練美國
開始訓練泛太平洋
開始訓練拉丁美洲
開始訓練新興亞洲
開始訓練新興歐洲
開始訓練歐洲
2013 2015 2015
開始訓練中國A
開始訓練東協
開始訓練美國
開始訓練泛太平洋
開始訓練拉丁美洲
開始訓練新興亞洲
開始訓練新興歐洲
開始訓練歐洲
     
##########開始選model並test data:
測試區間2017-03-01 00:00:00  2017-03-31 23:59:59.999999999
中國A的score為 0.9565217391304348
東協的score為 0.7391304347826086
美國的score為 0.21739130434782608
泛太平洋的score為 1.0
拉丁美洲的score為 0.5217391304347826
新興亞洲的score為 1.0
新興歐洲的score為 1.0
歐洲的score為 0.782608695652174
測試區間2017-04-01 00:00:00  2017-04-30 23:59:59.999999999
中國A的score為 0.2
東協的score為 1.0
美國的score為 0.05
泛太平洋的score為 0.35
拉丁美洲的score為 0.0
新興亞洲的score為 0.75
新興歐洲的score為 0.6
歐洲的score為 0.8
測試區間2017-05-01 00:00:00  2017-05-31 23:59:59.999999999
中國A的score為 0.043478260869565216
東協的score為 0.6086956521739131
美國的score為 1.0
泛太平洋的score為 0.5217391304347826
拉丁美洲的score為 0.5217391304347826
新興亞洲的score為 0.0
新興歐洲的score為 0.21739130434782608
歐洲的score為 0.9565217391304348
測試區間2017-06-01 00:00:00  

In [ ]:
'''
#算每個月各資產return
#test period  
period_test=pd.period_range('3/1/2017','11/1/2020',freq='M')

import pandas as pd
import numpy as np
import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")



country_dic={'中國A':'SHCOMP Index',
             '東協':'MXSO Index',
             '美國':'SPX Index',
             '泛太平洋':'MXAP Index',
             '拉丁美洲':'MXLA Index',
             '新興亞洲':'MXMS Index',
             '新興歐洲':'MXMU Index',
             '歐洲':'SX5P Index',
                    }

#創建空白dataframe
df_gain=pd.DataFrame(columns=[n for n in country_dic.keys()])

count=0
for i in range(0,len(period_test)-2,1):

    print("測試區間{0}  {1}".format(period_test[i].start_time,period_test[i].end_time))

    test_start=period_test[i].start_time
    test_end=period_test[i].end_time 
    real_start=period_test[i+2].start_time
    real_end=period_test[i+2].end_time
    for country in country_dic.keys():
        (df_price,indexes)=get_price(subject=country)
        df_price=df_price[test_end:real_end]
        gain=df_price[-1]-df_price[0]
        #存成dataframe~proba     
        df_gain.loc[count,country]=gain
        df_gain.loc[count,'date']=test_start 
    count=count+1   
    
#存檔
writer=pd.ExcelWriter('C:/Users/Z00045186/Desktop/國家_economic model/gain.xlsx')
df_gain.to_excel(writer,'outcome')
writer.save()    

In [ ]:
#檢查 dataset 是否一致
#dataset1218=dataset_1218[:'2020/12/18']
#dataset1228=dataset_1228[:'2020/12/18']
#
#differ_number=0
#for features in dataset1218.columns:
#    suma=0
#    for i in range(0,len(dataset1218[features])-1,1):
#        if dataset1218[features][i]!=dataset1228[features][i]:
#            suma=suma+1
#    if suma>30:     
#        differ_number=differ_number+1
#            
#        
#
#differ_number